##Upload database

In [22]:
from google.colab import files
uploaded = files.upload()


Saving chocolate_sales.db to chocolate_sales (1).db


##Create Table and Insert data

In [23]:
from sqlalchemy import Column, Integer, String, Float, ForeignKey, Date, create_engine
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

# 1. สร้างฐานข้อมูลใหม่
DATABASE_URL = "sqlite:///chocolate_sales(1).db"
engine = create_engine(DATABASE_URL, echo=True)

# 2. สร้าง Base
Base = declarative_base()

# 3. สร้าง Models
class SalesPerson(Base):
    __tablename__ = 'sales_person'
    sales_person_id = Column(Integer, primary_key=True)
    name = Column(String(100))
    country = Column(String(50))
    sales = relationship('Sale', back_populates='sales_person')

class Product(Base):
    __tablename__ = 'product'
    product_id = Column(Integer, primary_key=True)
    product_name = Column(String(100))
    sales = relationship('Sale', back_populates='product')

class Sale(Base):
    __tablename__ = 'sale'
    sale_id = Column(Integer, primary_key=True)
    sales_person_id = Column(Integer, ForeignKey('sales_person.sales_person_id'))
    product_id = Column(Integer, ForeignKey('product.product_id'))
    date = Column(Date)
    amount = Column(Float)
    boxes_shipped = Column(Integer)

    sales_person = relationship('SalesPerson', back_populates='sales')
    product = relationship('Product', back_populates='sales')

# 4. สร้างตารางในฐานข้อมูล
Base.metadata.create_all(engine)

from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

2025-04-27 01:35:56,310 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-04-27 01:35:56,312 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sales_person")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("sales_person")


2025-04-27 01:35:56,314 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:
CREATE TABLE sale (
	sale_id INTEGER NOT NULL, 
	sales_person_id INTEGER, 
	product_id INTEGER, 
	date DATE, 
	amount FLOAT, 
	boxes_shipped INTEGER, 
	PRIMARY KEY (sale_id), 
	FOREIGN KEY(sales_person_id) REFERENCES sales_person (sales_person_id), 
	FOREIGN KEY(product_id) REFERENCES product (product_id)
)




2025-04-27 01:35:56,372 INFO sqlalchemy.engine.Engine [no key 0.00172s] ()


INFO:sqlalchemy.engine.Engine:[no key 0.00172s] ()


2025-04-27 01:35:56,383 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [24]:
import pandas as pd

df = pd.read_csv('/content/Chocolate Sales.csv')

In [25]:
# หาคนขายไม่ซ้ำ
unique_sales_persons = df[['Sales Person', 'Country']].drop_duplicates()

# หาสินค้าไม่ซ้ำ
unique_products = df[['Product']].drop_duplicates()


In [26]:
# Insert Sales Person
sales_person_objects = [
    SalesPerson(name=row['Sales Person'], country=row['Country'])
    for _, row in unique_sales_persons.iterrows()
]
session.add_all(sales_person_objects)
session.commit()

# Insert Product
product_objects = [
    Product(product_name=row['Product'])
    for _, row in unique_products.iterrows()
]
session.add_all(product_objects)
session.commit()


2025-04-27 01:36:10,240 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-04-27 01:36:10,247 INFO sqlalchemy.engine.Engine INSERT INTO sales_person (name, country) VALUES (?, ?) RETURNING sales_person_id


INFO:sqlalchemy.engine.Engine:INSERT INTO sales_person (name, country) VALUES (?, ?) RETURNING sales_person_id


2025-04-27 01:36:10,249 INFO sqlalchemy.engine.Engine [generated in 0.00048s (insertmanyvalues) 1/150 (ordered; batch not supported)] ('Jehu Rudeforth', 'UK')


...

INFO:sqlalchemy.engine.Engine:INSERT INTO product (product_name) VALUES (?) RETURNING product_id


2025-04-27 01:36:11,049 INFO sqlalchemy.engine.Engine [insertmanyvalues 22/22 (ordered; batch not supported)] ('70% Dark Bites',)


INFO:sqlalchemy.engine.Engine:[insertmanyvalues 22/22 (ordered; batch not supported)] ('70% Dark Bites',)


2025-04-27 01:36:11,053 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [27]:
# สร้าง dictionary เอาไว้ mapping ชื่อ => id
sales_person_id_map = {sp.name: sp.sales_person_id for sp in session.query(SalesPerson).all()}
product_id_map = {p.product_name: p.product_id for p in session.query(Product).all()}


2025-04-27 01:36:21,288 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-04-27 01:36:21,293 INFO sqlalchemy.engine.Engine SELECT sales_person.sales_person_id AS sales_person_sales_person_id, sales_person.name AS sales_person_name, sales_person.country AS sales_person_country 
FROM sales_person


INFO:sqlalchemy.engine.Engine:SELECT sales_person.sales_person_id AS sales_person_sales_person_id, sales_person.name AS sales_person_name, sales_person.country AS sales_person_country 
FROM sales_person


2025-04-27 01:36:21,294 INFO sqlalchemy.engine.Engine [generated in 0.00165s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00165s] ()


2025-04-27 01:36:21,301 INFO sqlalchemy.engine.Engine SELECT product.product_id AS product_product_id, product.product_name AS product_product_name 
FROM product


INFO:sqlalchemy.engine.Engine:SELECT product.product_id AS product_product_id, product.product_name AS product_product_name 
FROM product


2025-04-27 01:36:21,302 INFO sqlalchemy.engine.Engine [generated in 0.00181s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00181s] ()


In [28]:
from datetime import datetime

# เตรียมข้อมูล Sale
sale_objects = []
for _, row in df.iterrows():
    sale = Sale(
        sales_person_id = sales_person_id_map[row['Sales Person']],
        product_id = product_id_map[row['Product']],
        date = datetime.strptime(row['Date'], "%d-%b-%y"),  # รูปแบบวันที่
        amount = float(row['Amount'].replace('$','').replace(',','')),  # แปลง amount เป็นตัวเลข
        boxes_shipped = int(row['Boxes Shipped'])
    )
    sale_objects.append(sale)

# Bulk insert
session.add_all(sale_objects)
session.commit()


2025-04-27 01:36:24,927 INFO sqlalchemy.engine.Engine INSERT INTO sale (sales_person_id, product_id, date, amount, boxes_shipped) VALUES (?, ?, ?, ?, ?) RETURNING sale_id


INFO:sqlalchemy.engine.Engine:INSERT INTO sale (sales_person_id, product_id, date, amount, boxes_shipped) VALUES (?, ?, ?, ?, ?) RETURNING sale_id


2025-04-27 01:36:24,929 INFO sqlalchemy.engine.Engine [generated in 0.00398s (insertmanyvalues) 1/1094 (ordered; batch not supported)] (143, 1, '2022-01-04', 5320.0, 180)


INFO:sqlalchemy.engine.Engine:[generated in 0.00398s (insertmanyvalues) 1/1094 (ordered; batch not supported)] (143, 1, '2022-01-04', 5320.0, 180)


2025-04-27 01:36:24,931 INFO sqlalchemy.engine.Engine INSERT INTO sale (sales_person_id, product_id, date, amount, boxes_shipped) VALUES (?, ?, ?, ?, ?) RETURNING sale_id


...

INFO:sqlalchemy.engine.Engine:INSERT INTO sale (sales_person_id, product_id, date, amount, boxes_shipped) VALUES (?, ?, ?, ?, ?) RETURNING sale_id


2025-04-27 01:36:28,059 INFO sqlalchemy.engine.Engine [insertmanyvalues 1094/1094 (ordered; batch not supported)] (106, 22, '2022-05-23', 5075.0, 344)


INFO:sqlalchemy.engine.Engine:[insertmanyvalues 1094/1094 (ordered; batch not supported)] (106, 22, '2022-05-23', 5075.0, 344)


2025-04-27 01:36:28,095 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


##export to csv

In [29]:
import sqlite3
import pandas as pd

# เชื่อมต่อ database
conn = sqlite3.connect('chocolate_sales(1).db')

# SQL Query
query = """
WITH TotalSalesPerPerson AS (
    SELECT
        sales_person_id,
        SUM(amount) AS total_sales_person
    FROM
        sale
    GROUP BY
        sales_person_id
),
RankedSales AS (
    SELECT
        sp.name AS sales_person_name,
        t.total_sales_person,
        p.product_name,
        SUM(s.amount) AS total_product_sales,
        ROW_NUMBER() OVER (PARTITION BY sp.name ORDER BY SUM(s.amount) DESC) AS rank_in_person
    FROM
        sale s
    JOIN
        sales_person sp ON s.sales_person_id = sp.sales_person_id
    JOIN
        product p ON s.product_id = p.product_id
    JOIN
        TotalSalesPerPerson t ON s.sales_person_id = t.sales_person_id
    GROUP BY
        sp.name, t.total_sales_person, p.product_name
)

SELECT
    sales_person_name,
    total_sales_person,
    product_name,
    total_product_sales
FROM
    RankedSales
WHERE
    rank_in_person <= 5
ORDER BY
    sales_person_name, total_product_sales DESC;
"""

# รัน query และเอาผลลัพธ์มาเก็บใน DataFrame
df = pd.read_sql_query(query, conn)

# export เป็น CSV
df.to_csv('top5_chocolate_per_salesperson.csv', index=False)
